In [4]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
html=urlopen('https://en.wikipedia.org/wiki/Comparison_of_text_editors')
bs=BeautifulSoup(html, 'html.parser')
table=bs.find_all('table', {'class', 'wikitable'})[0]
rows=table.find_all('tr')
csvFile=open('c:/data/csv/editors.csv','w',newline='',encoding='utf-8')
writer=csv.writer(csvFile)
try:
    for row in rows:
        csvRow=[]
        for cell in row.find_all(['td', 'th']):
            csvRow.append(cell.text.strip())
        writer.writerow(csvRow)
finally:
    print('saved')
    csvFile.close()

saved


In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import re
import random
import pymysql

conn = pymysql.connect(
    host="localhost", port=3306, user="web", password="1234", db="myweb", charset="utf8"
)
cursor = conn.cursor()
cursor.execute("use myweb")
cursor.execute("delete from pages")


def store(title, content):
    title = title.replace("'", "''")
    content = content.replace("'", "''")
    sql = "insert into pages (title, contect) values ('%s', '%s')" % (title, content)
    cursor.execute(sql)
    conn.commit()


def getLinks(url):
    html = urlopen("http://ko.wikipedia.org" + url)
    obj = bs(html, "html.parser")
    title = obj.find("h1").text
    content = obj.find("div", {"id": "mw-content-text"}).find("p").text

    store(title, content)
    return obj.find("div", {"id": "bodyContent"}).find_all(
        "a", href=re.compile("^(/wiki/)((?!:).)*$")
    )


links = getLinks("/wiki/%EC%9E%90%EB%B0%94")
try:
    cnt = 0
    while True:
        cnt += 1
        if cnt >= 10:
            break
        newArticle = links[random.randint(0, len(links) - 1)].attrs["href"]
        print(newArticle)
        links = getLinks(newArticle)

finally:
    cursor.close()
    conn.close()
    print("complete")

len(links)

OperationalError: (1054, "Unknown column 'contect' in 'field list'")